We're going to look into the Runnable - one of the core LangChain primitives, and how to use LCEL (LangChain Expressive Language).

# LangChain Runnables

In [2]:
!pip install langchain-core

  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp311-cp311-macosx_11_0_arm64.whl (1.8 MB)
Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl (172 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_core.runnables import RunnableLambda

Let's start with a very simple *Runnable* created from a function:

In [4]:
runnable = RunnableLambda(lambda x: x + 1)

runnable.invoke(1)

2

Now we can put together our first chain:

In [5]:
from typing import Optional
from langchain_core.runnables import Runnable, RunnableConfig

def increment_by_one(x: int) -> int:
  return x + 1

def fake_llm(x: int) -> str:
  return f"Result = {x}"


class MyFirstChain(Runnable[int, str]):

   def invoke(self, input: str, config: Optional[RunnableConfig] = None) -> str:
    increment = increment_by_one(input)
    return fake_llm(increment)


In [6]:
chain = MyFirstChain()
result = chain.invoke(1)
print(result)

Result = 2


And we can do the same much easier with LangChain Expressive Language (LCEL):

In [7]:
chain = (
    RunnableLambda(increment_by_one)
    | RunnableLambda(fake_llm)
)


result = chain.invoke(1)
print(result)

Result = 2


Actually, you should only convert the last element explicitly to RunnableLambda, and LangChain would take care about else for you:

In [8]:
chain = (
    increment_by_one
    | RunnableLambda(fake_llm)
)


result = chain.invoke(1)
print(result)

Result = 2


LCEL with | operator is actually equivalent to creating a RunnableSequence explicitly:

In [9]:
from langchain_core.runnables import RunnableSequence


a = (RunnableLambda(increment_by_one) | RunnableLambda(fake_llm))
b = RunnableSequence(RunnableLambda(increment_by_one), RunnableLambda(fake_llm))

a == b

True

## RunnableParallel

Another powerful LCEL primitive is RunnableParallel. You pass multiple chains as named arguments, and it runs them in parallel and combines their outputs into a dict with keys being argnames and values being outputs:

In [10]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(step1=increment_by_one | RunnableLambda(fake_llm), step2=fake_llm)

chain.invoke(1)

{'step1': 'Result = 2', 'step2': 'Result = 1'}

You can also easily compose chains:

In [11]:
chain1 = increment_by_one | chain
chain1.invoke(1)

{'step1': 'Result = 3', 'step2': 'Result = 2'}

You don't need to use the RunnableParallel constructor, you can just combine the chains within a dictionary:

In [12]:
from langchain_core.runnables import RunnableParallel

chain2 = ( RunnableLambda(increment_by_one)
  | {"step1": increment_by_one | RunnableLambda(fake_llm), "step2": fake_llm}
)

chain2.invoke(1)

{'step1': 'Result = 3', 'step2': 'Result = 2'}

In [13]:
print(chain1 == chain2)

True


## itemgetter

We typically pass input as dictionaries, and there's a convinient way to retrieve an element from a dictionary with a built-in *itemgetter* function:

In [14]:
from operator import itemgetter

chain = (
  itemgetter("x")
  | RunnableLambda(increment_by_one)
  | fake_llm
)


chain.invoke({"x": 1})

'Result = 2'

## RunnablePassThrough

And we can modify dictionaries in-place with Runnables (by assigning values into a dictionary or create new dictionaries). That's how we produce an output dictionary:

In [15]:
from langchain_core.runnables import RunnablePassthrough

chain_rps = RunnableParallel(
    origin=RunnablePassthrough(),
    output=increment_by_one
)

chain_rps.invoke(1)

{'origin': 1, 'output': 2}

Now let's create a new dictionary by adding (assigning) additional values to it:

In [16]:
chain_assign = RunnablePassthrough().assign(y=itemgetter("x") | RunnableLambda(increment_by_one))

query = {"x": 1}
result = chain_assign.invoke({"x": 1})
print(result)

{'x': 1, 'y': 2}


And let's put it all together by creating a more complex dictionary (with sub-chains):

In [17]:
chain_rps = RunnableParallel(
    origin=RunnablePassthrough().assign(length=lambda input: input["x"]+1),
    modified=lambda input: increment_by_one(input["x"])
)

chain_rps.invoke({"x": 1})

{'origin': {'x': 1, 'length': 2}, 'modified': 2}